In [3]:
!python --version

Python 3.7.12


In [5]:
import pandas as pd
import numpy as np
from cvxpy import *
from google.colab import files
import matplotlib.pyplot as plt
import io
plt.style.use('seaborn-darkgrid')

In [11]:
uploaded = files.upload()

Saving latest_monthly_prices.csv to latest_monthly_prices (1).csv
Saving monthly_prices.csv to monthly_prices (1).csv


# Original Monthly Closing Prices

In [8]:
"""Code below is slightly modified from what was provided to work in Colab"""

def main(filename):
  # read monthly closing prices
  mp = pd.read_csv(io.BytesIO(uploaded[filename]), index_col=0)
  mr = pd.DataFrame()

  # compute monthly returns
  for s in mp.columns:
      date = mp.index[0]
      pr0 = mp[s][date] 
      for t in range(1,len(mp.index)):
          date = mp.index[t]
          pr1 = mp[s][date]
          ret = (pr1-pr0)/pr0
          #mr.set_value(date,s,ret)
          mr.at[date,s]=ret
          pr0 = pr1
          
  # get symbol names
  symbols = mr.columns

  # convert monthly return data frame to a numpy matrix
  #return_data = mr.as_matrix().T
  return_data = mr.values.T

  # compute mean return
  r = np.asarray(np.mean(return_data, axis=1))

  # covariance
  C = np.asmatrix(np.cov(return_data))

  # print out expected return and std deviation
  print("----------------------")
  for j in range(len(symbols)):
      print('%s: Exp ret = %f, Risk = %f' %(symbols[j],r[j], C[j,j]**0.5))
    

  # set up optimization model
  n = len(symbols)
  x = Variable(n)
  req_return = 0.02
  ret = r.T*x
  risk = quad_form(x, C)
  prob = Problem(Minimize(risk), 
                [sum(x) == 1, ret >= req_return, x >= 0])

  # solve problem and write solution
  try:
      prob.solve()
      print("----------------------")
      print("Optimal portfolio")
      print("----------------------")
      for s in range(len(symbols)):
          #print('x[%s] = %f'%(symbols[s],x.value[s,0]))
          print('x[%s] = %f'%(symbols[s],x.value[s]))
      print("----------------------")
      print('Exp ret = %f' %(ret.value))
      print('risk    = %f' %((risk.value)**0.5))
      print("----------------------")
  except:
      print('Error')

main("monthly_prices.csv")

----------------------
MSFT: Exp ret = 0.024611, Risk = 0.058040
V: Exp ret = 0.018237, Risk = 0.042807
WMT: Exp ret = 0.009066, Risk = 0.044461
----------------------
Optimal portfolio
----------------------
x[MSFT] = 0.582818
x[V] = 0.204324
x[WMT] = 0.212858
----------------------
Exp ret = 0.020000
risk    = 0.038256
----------------------


# January 2020 - December 2021 Closing Prices

In [12]:
main("latest_monthly_prices.csv")

----------------------
MSFT: Exp ret = 0.032716, Risk = 0.063093
V: Exp ret = 0.007660, Risk = 0.084906
WMT: Exp ret = 0.012749, Risk = 0.051777
----------------------
Optimal portfolio
----------------------
x[MSFT] = 0.380516
x[V] = 0.068093
x[WMT] = 0.551391
----------------------
Exp ret = 0.020000
risk    = 0.047298
----------------------
